# 8.2 Web Scraping I.


Web-Scrapping es la forma que tenemos para referirnos a la captura de información de cualquier sitio web. Su objetivo es capturar información de forma automática.

Las librerías principales que vamos a utilizar son beautifulsoup y requests. Para ello, hay que instalarlas primero:


In [ ]:
!pip install beautifulsoup4
!pip install requests

Una vez instalado, se importa en el código:

In [ ]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

El proceso se divide principalmente en tres fases:

- Carga de la dirección web a la que realizar el scrapping. A través de  requests.

- Extracción del contenido de la web a partir de Beautifulsoup

- Manipulación del contenido.


## 1. Carga de la URL

Request necesita dos cosas: La URL y un elemento cabecera. El elemento cabecera es vital, ya que muchas web no permiten el acceso a la información a no ser que detecten que las peticiones vienen de un navegador. El más utilizado para este propósito es Mozilla Firefox.

Así, definimos headers como un diccionario cuyo User-Agent es Firefox:

In [ ]:
headers = {'User-Agent': 'Firefox'}
url = 'https://resultados.as.com/resultados/futbol/primera/2015_2016/jornada/regular_a_1/'

In [ ]:
re = requests.get(url,headers=headers)

Ahora debemos preguntar si la conexion con esa pagina web ha funcionado. Es decir si la propiedad status_code que devuelve re es 200 o 201. Si es así, pasamos a la siguiente fase.

In [ ]:
print(re.status_code)

En re.content tendremos todo el HTML

In [ ]:
re.content

## 2. Extracción del contenido con BeautifulSoup

Lo primero que tenemos que hacer es pasar al contenido que se encuentra en re.text a través de un parser de HTML.

In [ ]:
soup = BeautifulSoup( re.text, 'html.parser')

Lo siguiente que tenemos que hacer es identificar qué información es la que nos queremos descargar, y dónde se encuentra dentro del HTML

Todos los elementos de una página tienen un xpath que es unívoco a ese elemento, por lo que puedes usarlo para recuperar la información

- Inspeccionar, botón derecho sobre el elemento, copiar, copiar XPath

<center>
<img src="./imgs/xpath.png"  alt="drawing" width="600"/>
</center>

Para el ejemplo, vamos a descargarnos la información de la jonada de fútbol de LaLiga Santander

<center>
<img src="./imgs/info_a_descargar.png"  alt="drawing" width="600"/>
</center>

Cada uno de los resultados tienen la etiqueta li, y la clase list-resultado. Por lo que localizamos todos los elementos que cumplan estas condiciones y los guardamos

<center>
<img src="./imgs/ejemplo_1.png"  alt="drawing" width="600"/>
</center>

In [ ]:
articles = soup.find_all('li',{'class' : 'list-resultado'})

Ya los tenemos todos. Vamos a ver qué pinta tienen

In [ ]:
articles

No es precisamente información limpia. 

Tendremos que iterar por los artículos para obtener la información que nos interesa

In [ ]:
for article in articles:
    
    equipos = article.find_all('span',{'class' : 'nombre-equipo'})
    
    for equipo in equipos:
        
        eq =equipo.text
        print(eq)
    
    resultado = article.find('div',{'class' : 'cont-resultado finalizado'}).getText()
    print(resultado)

Podemos acceder a cada elemento con el nombre del tag

In [ ]:
soup.head

Si somos específicos, obtendremos mejores resultados

In [ ]:
soup.head.title

Con .text o .string accedemos al contenido

In [ ]:
soup.head.title.text

## 3. Manipulación del contenido

En esta fase vamos iterando sobre lo que hemos conseguido con BeautifulSoup.

Si el contenido es directamente la información que se encuentra en lo encontrado por soup.find.all, sólo hay que imprimir la información, o guardarla en un DF.

Sin embargo a veces hay que volver a llamar volver a repetir la búsqueda con find, itereando sobre la información descargada y filtrada

In [ ]:
for article in articles:

    masinfo= article.find_all('div',{'class' : 'info-evento'})

    for info in masinfo:
        fechahora = info.find('time').get('content')
        print(fechahora)

En el anterior código, vemos que para obtener la fecha y la hora simplemente utilizamos find. Este es el caso cuando no hay una lista de elementos. Suele devolver un dato ya disponible, no un objeto iterable.

Otra alternativa que podemos probar cuando este get no funcione porque necesitemos especificar la clase es:

    resultado = enlace.find('a',{'class' : 'resultado'}).getText()

## Conclusión

El proceso de Web-Scrapping no es un proceso complicado, pero si tedioso.
Y es tedioso porque hay que comprender cuál es la estructura de la web que queremos scrappear y es posible que con el tiempo, un web-scrapper que funcionase, no nos funcione actualmente por qu hayan cambiado la estructura de la web.

Para asentar conocimientos, vamos a probar a extraer la misma información que ya obtuvimos haciendo este mismo proceso en R

<center>
<img src="./imgs/wikipedia.png"  alt="drawing" width="600"/>
</center>

Los primeros pasos son exáctamente iguales

Lo único que tenemos que adaptar es la url

In [ ]:
headers = {'User-Agent': 'Firefox'}
url = 'https://es.wikipedia.org/wiki/El_lobo_de_Wall_Street'

In [ ]:
re = requests.get(url,headers=headers)

In [ ]:
soup = BeautifulSoup( re.text, 'html.parser')

Capturamos la ficha técnica de la película (la tabla que está a la derecha)

In [ ]:
datos_a_extraer = soup.find_all('table',{'class' : 'infobox plainlist plainlinks'})

Es la primera tabla, por lo que debemos indicar que nos quedamos con únicamente esa tabla

Las filas de la tabla tienen la etiqueta tr, por lo que localizamos todos los elementos que tengan dicha etiqueta, para iterar sobre ellos

In [ ]:
datos_a_extraer = datos_a_extraer[0]
trs = datos_a_extraer.find_all('tr')

Al iterar sobre las filas, queremos extraer los elementos de las columnas de la izquierda th y de la derecha td

Extraemos todos e iteramos sobre ellos para extraer la información

In [ ]:
rows_th = []
rows_td = []

for tr in trs:
    
    ths = tr.find_all('th')
    tds = tr.find_all('td')
    
    for th in ths:
                        
        texto = th.text
        # print(th.text)
        rows_th.append(texto)
        
    for td in tds:
                
        texto = td.text
        #print(td.text)
        rows_td.append(texto)

In [ ]:
resultado = pd.DataFrame(np.column_stack([rows_th, rows_td]))
resultado

Sin embargo, si analizamos el resultado obtenido, no es exáctamente lo que queremos.

La información está descolocada. Y eso es porque hay varios elementos en la tabla que solo están en una de las dos columnas.

Tenemos que identificarlos, y evitar descargarlos

<center>
<img src="./imgs/tabla_mal.png"  alt="drawing" width="600"/>
</center>

In [ ]:
rows_th = []
rows_td = []

for tr in trs:
    
    ths = tr.find_all('th')
    tds = tr.find_all('td')
    
    for th in ths:
        
        if "Ficha técnica" in str(th) or "Datos y cifras" in str(th) or "Compañías" in str(th):
            continue
        
        texto = th.text
        # print(th.text)
        rows_th.append(texto)
        
    for td in tds:
        
        if "Ver todos los créditos" in str(td) or "Ficha" in str(td) or "Wikidata" in str(td):
            continue
        
        texto = td.text
        texto = texto.replace('\n',"")
        #print(td.text)
        rows_td.append(texto)

In [ ]:
resultado = pd.DataFrame(np.column_stack([rows_th, rows_td]))
resultado

Ahora sí que lo hemos conseguido

Capturamos ahora las referencias de la película

Ya tenemos descargado el contenido en soup

Lo que tenemos que hacer es localizar el elemento y clase por el que queremos filtrar

En este caso, ol y references

In [ ]:
datos_a_extraer = soup.find_all('ol',{'class' : 'references'})
datos_a_extraer = datos_a_extraer[0]

Vemos que cada una de las referencias tiene un tag que se llama li. 

Localizamos todas e iteramos por ellos.

In [ ]:
lis = datos_a_extraer.find_all('li')

In [ ]:
rows = []

for info in lis:
    texto = info.find('span',{'class' : 'reference-text'}).getText()    
    # print(texto)
    rows.append(texto)

Convertimos la información en un dataframe y ya lo tenemos

In [ ]:
df = pd.DataFrame(rows)
df.columns = ['Referencias']
df

Capturemos ahora el argumento de la película

El elemento por el que queremos filtrar es div, y la clase mw-parser-output

In [ ]:
datos_a_extraer = soup.find_all('div',{'class' : 'mw-parser-output'})
datos_a_extraer = datos_a_extraer[0]

Cada uno de los párrafos tienen la etiqueta p

Localizamos todos e itereamos por ellos

In [ ]:
lis = datos_a_extraer.find_all('p')

In [ ]:
rows = []

for li in lis:
    
    texto = li.text
    #print(texto)
    rows.append(texto)

In [ ]:
df = pd.DataFrame(rows)
df.columns = ['Argumento']
df = df.iloc[2:-3,:]

df

## Ejercicios

**8.2.1** Captura los hechos relevantes de la CNMV: http://www.cnmv.es/portal/HR/HRAldia.aspx

**8.2.2** Captura las noticias sobre economía de Expansion: http://www.expansion.com/economia.html?intcmp=MENUHOM24101&s_kw=economia

**8.2.3** Captura la agenda macroeconómica de investing.com: http://es.investing.com/economic-calendar/

**8.2.4** Captura las cotizaciones de los commodities de investing.com: https://es.investing.com/commodities/

**8.2.5** Captura las cotizaciones del Ibex 35 de Bolsa de Madrid: http://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000

**8.2.6** De la página de bolsa de madrid: https://www.bolsamadrid.es/esp/aspx/Indices/Resumen.aspx obten un daframe con los contenidos de la tabla.

**8.2.7** Obtén las últimas noticias (expansión) y precios objetivo (expansión) de: Telefónica, Santander y Siemens Gamesa

Noticias: 

- https://www.expansion.com/mercados/cotizaciones/valores/telefonica_M.TEF.html
- https://www.expansion.com/mercados/cotizaciones/valores/santander_M.SAN.html
- https://www.expansion.com/mercados/cotizaciones/valores/siemensgamesa_M.SGRE.html

Precios objetivo: 

- https://www.expansion.com/mercados/bolsa/recomendaciones/consenso-mercados/t/telefonica_M.TEF.html
- https://www.expansion.com/mercados/bolsa/recomendaciones/consenso-mercados/s/santander_M.SAN.html
- https://www.expansion.com/mercados/bolsa/recomendaciones/consenso-mercados/s/siemensgamesa_M.SGRE.html

**8.2.8** Desarrolla una función que sea capaz de capturar todas las noticias que tengan relación con una cualquier empresa y que hayan tenido lugar en un intervalo determinado. Utiliza para ello la hemeroteca de ABC. Pueba a obtener todas la noticias de Telefónica entre el 1 de enero y 31 de enero de 2016.

- http://www.abc.es/hemeroteca/resultados-busqueda-avanzada/noticia/pagina-1?exa=telefonica&rfec=20160101;20161031&or=1&nres=20